# Diluted Unitary Fitting

In [28]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from scipy.stats import gaussian_kde
from quantum_circuits import *
from analysis import *
from utils import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [32]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              rank_start = 1, 
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001): 
    
    model_list = []
    distance_list = []

    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_haar_random(d, rng=rng), 
                                                          c=0.5, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = AnnulusDistance(),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = AnnulusDistance(), 
                                            verbose=True,),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    
                   )

        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        idx = np.argmin(distance_list)
        
        if distance_list[idx]<distance_list[-1]:
            print("Done!")
            break
            
    
    idx = np.argmin(distance_list)
    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step,
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        U = generate_haar_random(d, rng) 
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Three Qubits

In [33]:
[channel_list1, channel_list2, channel_list3, channel_list4, channel_list5, channel_list6, channel_list7] = loader(data_path("channel_fullRank_belem_longTrain.p"))

spectrum_target_list1 = [channel_spectrum(channel) for channel in channel_list1]
spectrum_target_list2 = [channel_spectrum(channel) for channel in channel_list2]
spectrum_target_list3 = [channel_spectrum(channel) for channel in channel_list3]
spectrum_target_list4 = [channel_spectrum(channel) for channel in channel_list4]
spectrum_target_list5 = [channel_spectrum(channel) for channel in channel_list5]
spectrum_target_list6 = [channel_spectrum(channel) for channel in channel_list6]
spectrum_target_list7 = [channel_spectrum(channel) for channel in channel_list7]

In [34]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list1, 
                                                          d=d, 
                                                          rank_start=1, 
                                                          rank_step=1,
                                                          num_iter = 2000,
                                                         )

saver([distance_best_list, model_best_list], data_path("diluted_fitting_8layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.30029435965748985 None
0.24843003403904626 None
0.2202199990690521 None
0.16823157883473425 None
0.15293211741801843 None
0.1103648100208441 None
0.08480811428082641 None
0.061462366206407264 None
0.039330009131291165 None
0.03792735097096872 None
0.03792474865640967 None
0.03789594510112941 None
0.0379138591746508 None
0.0378999080499072 None
0.03789909891858711 None
0.037899678974328596 None
0.03790742078663499 None
0.037906390360627436 None
0.03792107297942284 None
0.037901971169075135 None
0.03789781368769146 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.2569890915237481 None
0.20735266959926896 None
0.1912275946207169 None
0.1611843979706479 None
0.11696301464318938 None
0.09009468146425575 None
0.06431417147435653 None
0.039692421252050025 None
0.01633987311093453 None
0.028498680145849126 None
0.028461547439450768 None
0.028441936985386686 None
0.012138388475845559 None
0.012151684723391613 None
0.012145060302157987 None
0.01212624255639093 None
0.028450259727554425 None
0.012179208305582123 None
0.01215606851296032 None
0.012136731247666864 None
0.012127354044919813 None


...

0.2547386670851487 None
0.22275569771827114 None
0.20800297592547934 None
0.16184935617979432 None
0.13362506927057258 None
0.10683847098912456 None
0.08107460555493232 None
0.056530728019616845 None
0.033170405982603395 None
0.016027921502388895 None
0.015900844067951152 None
0.015887748688935444 None
0.015902876376718805 None
0.015884929427226985 None
0.015889046164914492 None
0.015887484690274813 None
0.015905282034759417 None
0.015879153906568043 None
0.015889222218442203 None
0.01588455025303322 None
0.01589464638939721 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.24828433416000123 None
0.23216146982679473 None
0.1858930512446473 None
0.15588213146094285 None
0.12713155276475915 None
0.09992420527225888 None
0.07433332107613752 None
0.049831771711118425 None
0.02675358228113832 None
0.00706901543756416 None
0.006502174109023131 None
0.006517739037796636 None
0.006488947381798417 None
0.006495969928582225 None
0.006482028738213194 None
0.006498464462802871 None
0.006517194963189196 None
0.006512436154850982 None
0.006491025429106867 None
0.006510329923784474 None
0.006499384739585081 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.25118957174469203 None
0.2061583692434207 None
0.19402093491943365 None
0.16612212940424065 None
0.13895972488660768 None
0.09645167549791572 None
0.07127771603725848 None
0.06339966066665229 None
0.024013980451203322 None
0.008637492432521328 None
0.008589307880565302 None
0.00859023315902627 None
0.008623517116713175 None
0.008573949131357901 None
0.008579634185705745 None
0.008616922906933097 None
0.00859842950914281 None
0.008641281523693879 None
0.008609288557309107 None
0.00858240121412179 None
0.008601028299963922 None
Done!


In [35]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list2, 
                                                          d=d, 
                                                          rank_start=1, 
                                                          rank_step=1,
                                                          num_iter = 2000)

saver([distance_best_list, model_best_list], data_path("diluted_fitting_16layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.16142032802778997 None
0.16017077040653652 None
0.1435128787000542 None
0.1587896213588206 None
0.1433006384444445 None
0.14329846752817652 None
0.15873525657872392 None
0.1587352464665353 None
0.14329831342636226 None
0.14329831338052115 None
0.15873524589841037 None
0.15873524589873395 None
0.15873524589946444 None
0.1587352458989718 None
0.15873524589925503 None
0.15873524589959118 None
0.1432983133773239 None
0.15873524589901494 None
0.15873524589919943 None
0.14329831337594998 None
0.15873524589864835 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.062219490412483446 None
0.038353987878248966 None
0.05466983549977748 None
0.03834205278189678 None
0.054659012112926765 None
0.0547115208861466 None
0.05464354704964762 None
0.054661083987342776 None
0.054647124923410506 None
0.05467453291593034 None
0.054665930507932774 None
0.05466145411481751 None
0.03833725143866586 None
0.054655105564809396 None
0.05464699928906068 None
0.03835023977742003 None
0.054671531894735684 None
0.054716695963529655 None
0.038374694471815234 None
0.03833726891197475 None
0.054682831198770765 None


...

0.126032415480286 None
0.12557760204582863 None
0.11004946598656186 None
0.11001313288130211 None
0.11000697508599908 None
0.11000628571556612 None
0.12544806188334684 None
0.11000620921499109 None
0.12544805462222808 None
0.12544805456691796 None
0.12544805456288352 None
0.1100062089119142 None
0.1254480545630824 None
0.1100062089232676 None
0.1100062089178622 None
0.11000620891523324 None
0.12544805455968766 None
0.12544805456267 None
0.1254480545661524 None
0.11000620891807569 None
0.11000620891773237 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.07095876376601386 None
0.050810631102855844 None
0.035293304758001215 None
0.05074278690897148 None
0.05074178481388269 None
0.05074412392012806 None
0.050729267326363695 None
0.05075720416524218 None
0.035346478782656274 None
0.035332497292256126 None
0.05074998674318941 None
0.035288275479271686 None
0.035353829314341986 None
0.05077238594432025 None
0.05074002226031508 None
0.03531261722267124 None
0.03531335543608358 None
0.03531046270599081 None
0.03529392903505817 None
0.03531736830600127 None
0.03529369363277236 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.03972192478957297 None
0.023092059751165732 None
0.0231540549307784 None
0.038530142782363455 None
0.03854215592310474 None
0.023117831032713726 None
0.023155525884596368 None
0.023136750259642028 None
0.038634296407174876 None
0.03853637342235326 None
0.038557050914289244 None
0.03855790593599654 None
0.03852742049586252 None
0.023088187880406185 None
0.038554937720402464 None
0.038533553238446025 None
0.023091085463830447 None
0.023118380459844726 None
0.02316229054729233 None
0.023120733617316567 None
0.038541813877526004 None
Done!


In [36]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list3, 
                                                          d=d, 
                                                          rank_start=1, 
                                                          rank_step=1,
                                                          num_iter = 2000)

saver([distance_best_list, model_best_list], data_path("diluted_fitting_24layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.27600908037232097 None
0.281684524499661 None
0.28022479195359373 None
0.2636744937453674 None
0.2636361118872932 None
0.27993503866544783 None
0.27993503836526507 None
0.2636259804496675 None
0.26362598045032315 None
0.26362598044894686 None
0.2636259804492217 None
0.2636259804494512 None
0.2636259804489098 None
0.26362598044893953 None
0.2636259804499735 None
0.2636259804507484 None
0.27993503836417166 None
0.27993503836294104 None
0.27993503836296413 None
0.26362598045100527 None
0.2636259804493368 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.17735614358013274 None
0.14841736898096303 None
0.15571866382286867 None
0.13530185540833425 None
0.14899896158380396 None
0.1459318203075542 None
0.14077964256956121 None
0.12283918503320121 None
0.13968884860118364 None
0.1224438815393305 None
0.12247896415789268 None
0.1061104236523525 None
0.10612502576511655 None
0.13959687417740277 None
0.12242313450793382 None
0.10613893633520378 None
0.10611742309845867 None
0.12242295690408901 None
0.1227932669335598 None
0.10638443149653773 None
0.13977703232294575 None


...

0.07047295443194092 None
0.05080399218576448 None
0.0323006676310932 None
0.018723137230506854 None
0.018720711861143102 None
0.018737515536271256 None
0.01872416731189449 None
0.018694223080439974 None
0.01871956896839023 None
0.018700846145766736 None
0.01869814868346393 None
0.018707858409876633 None
0.018714374182227905 None
0.018698155409150978 None
0.018702054978815264 None
0.018705129266897703 None
0.018743343118575072 None
0.018694209601983123 None
0.018700664034339145 None
0.0187138019148456 None
0.018690681773315267 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.0634789465385206 None
0.03785290177305904 None
0.02028213777032124 None
0.007037453490981069 None
0.006974406541019709 None
0.0070188266540657535 None
0.006992896937298123 None
0.0069628808794918486 None
0.006974499517640631 None
0.006955184011935017 None
0.006979231801766967 None
0.006957512510897436 None
0.0069543962735377236 None
0.0069848923002334695 None
0.006978650538419143 None
0.007006716726163847 None
0.006994894476816453 None
0.006979667077461484 None
0.006990734432123939 None
0.006988226598886644 None
0.006955431496301014 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.07953226402758301 None
0.05478794599014963 None
0.03819466054587542 None
0.020806687333058998 None
0.016993760384340955 None
0.016924774024961512 None
0.01693275940594462 None
0.016923959642391302 None
0.016954296147776785 None
0.01695432924255951 None
0.01695902703135909 None
0.016922664403354375 None
0.016919798964558266 None
0.016938038554357774 None
0.0169275665646432 None
0.01698402950961827 None
0.01700342941409016 None
0.01694789105296963 None
0.016932098530201757 None
0.01698418370793709 None
0.016918842479293934 None
Done!


In [37]:
n = 3
d = 2**n

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)
rng = np.random.default_rng(seed=42)

distance_best_list, model_best_list = fit_diluted_unitary(spectrum_target_list4, 
                                                          d=d, 
                                                          rank_start=1, 
                                                          rank_step=1,
                                                          num_iter = 2000)

saver([distance_best_list, model_best_list], data_path("diluted_fitting_32layer.model"))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

0.37859516105568564 None
0.3656568192213088 None
0.3642599536639031 None
0.3803200355822406 None
0.36394276251895624 None
0.3802322600533332 None
0.380231426541675 None
0.3639223686190841 None
0.38023142653166153 None
0.380231426533052 None
0.3639223686192082 None
0.3802314265321264 None
0.38023142653290737 None
0.38023142653187214 None
0.38023142653205705 None
0.3802314265334155 None
0.36392236861892907 None
0.38023142653322245 None
0.3639223686198652 None
0.3639223686197302 None
0.38023142653249226 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.27765253174988863 None
0.24871378225948737 None
0.2396991336088613 None
0.2519148478232174 None
0.24929547274382186 None
0.26344029643005806 None
0.22475750424489332 None
0.22408130307751709 None
0.20690083156784664 None
0.22306096601830078 None
0.20675246723539212 None
0.20675352426408017 None
0.2067545085248777 None
0.22305928709317968 None
0.20676653558167335 None
0.22306005493777228 None
0.2230618419442302 None
0.20675392611768303 None
0.22305981695522847 None
0.22306151295223364 None
0.22305903589383577 None


...

0.3043895329105751 None
0.2759381341425099 None
0.2476392912626108 None
0.21961776065231797 None
0.19182369404822955 None
0.164433579435157 None
0.13740912540011807 None
0.12237679446224342 None
0.131190117866575 None
0.10794685064573627 None
0.11793376668339758 None
0.09593538427711434 None
0.10729959643048427 None
0.10297287209730853 None
0.09868753102651265 None
0.081079883032696 None
0.0762259918293205 None
0.07044257259394632 None
0.08581555695613788 None
0.06846991535981096 None
0.06845049035633909 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.3064814032660466 None
0.2941388134280957 None
0.25035682710020346 None
0.2382214594832306 None
0.1955748407467952 None
0.16908156338107333 None
0.1594720291159758 None
0.11942312410041767 None
0.11464304694839589 None
0.09113903256407431 None
0.08418116137656081 None
0.09426226912543405 None
0.07209026847177566 None
0.06662861345587374 None
0.06165726103862907 None
0.05683010456547359 None
0.051626599332629745 None
0.061682492869415975 None
0.06913800740262982 None
0.050725064802080416 None
0.06566928148873194 None


  0%|          | 0/2000 [00:00<?, ?it/s]

0.30919678803212886 None
0.29725268379979114 None
0.25306464624052516 None
0.22552671753429068 None
0.19833669843676013 None
0.17196260425060225 None
0.1627011826648156 None
0.12089588187257193 None
0.11192892411501587 None
0.08611845752398489 None
0.07824286292100707 None
0.07123494562040521 None
0.08211488364562494 None
0.060654638505070316 None
0.055720187984932744 None
0.05286779522861107 None
0.05286133298369081 None
0.06916272244301006 None
0.08632462280152006 None
0.06916272244289451 None
0.06916272243847288 None
Done!


## Four Qubits

In [38]:
n = 4
d = 2**n

[model1, model2, model3, model4, model5] = loader(data_path("belem_4qubit_repeated.model"))

spectrum1 = channel_spectrum(model1.channel, use_coords=True)
spectrum2 = channel_spectrum(model2.channel, use_coords=True) 
spectrum3 = channel_spectrum(model3.channel, use_coords=True)
spectrum4 = channel_spectrum(model4.channel, use_coords=True)
spectrum5 = channel_spectrum(model5.channel, use_coords=True)

In [39]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_8layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.2085285055390247 None
0.20772108047127816 None
0.20304760627128066 None
0.20304064670395125 None
0.2069356299367542 None
0.20304064668710342 None
0.2030406466871089 None
0.20693562993675918 None
0.2069356299367567 None
0.20693562993676387 None
0.2069356299367602 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.11264331762034509 None
0.0980024674459395 None
0.08987219399363602 None
0.08840771668118627 None
0.08463397391653817 None
0.08439493711056795 None
0.08047951703712536 None
0.08047595936629584 None
0.0843816241188914 None
0.08048799919145525 None
0.08048236400424513 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.07544832727050488 None
0.047983374209919605 None
0.020495751950615575 None
0.00266660494267037 None
0.002519459108687905 None
0.0024549374849748124 None
0.00248198623793877 None
0.0024772515921205646 None
0.002486610360585952 None
0.002481699596648453 None
0.002464453382341284 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.08038972484698108 None
0.04932932829120148 None
0.02583176606751496 None
0.0016415419940773344 None
0.00539654763815662 None
0.005381804959246181 None
0.009250882757283517 None
0.005411452103680733 None
0.0015054436387361438 None
0.005386699433998443 None
0.0015384608055065562 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.07564137349156974 None
0.04860920626157788 None
0.02161592917454029 None
0.0018700385480870058 None
0.001617774365076723 None
0.0016150190354173955 None
0.0016229007402945677 None
0.0016310892147336394 None
0.0016152285462323463 None
0.0016117318143597462 None
0.0016449867136785795 None
Done!


In [40]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_16layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.3529513921426881 None
0.345806609160031 None
0.3440198959077034 None
0.34376247216784356 None
0.3437408139234092 None
0.3397908991599398 None
0.34373910900830096 None
0.33979079231476966 None
0.3397907921962647 None
0.34373910447705985 None
0.34373910447704487 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.250967453470484 None
0.24350532361898186 None
0.23184137963658052 None
0.2247380449891482 None
0.22863315635506792 None
0.22863315635120013 None
0.2324753629364637 None
0.22863315635106185 None
0.22473803692417552 None
0.22863315635109657 None
0.2286331563511019 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.22813497407010586 None
0.20516523373941534 None
0.178303863784635 None
0.1516834018602287 None
0.12153802683019785 None
0.09582957478698785 None
0.07065075261252601 None
0.05357838714627559 None
0.04011512454082793 None
0.027357670603929515 None
0.01947512505756322 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.22846413978072966 None
0.19765559477213673 None
0.17483410438178562 None
0.14834916218940206 None
0.12230124385554668 None
0.09269109556431822 None
0.06767418285884577 None
0.050173549896082296 None
0.03703190008798793 None
0.024663674944777513 None
0.009173463758337967 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.2305143131514942 None
0.2036806260601801 None
0.17689531040916348 None
0.1503958122956705 None
0.12034694118531288 None
0.09476613804492098 None
0.0738847379425756 None
0.05213050372418569 None
0.0385508245308883 None
0.025591005072525992 None
0.013467920069588461 None
Done!


In [41]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_24layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.4539175078479235 None
0.45174514281701017 None
0.45165340280746924 None
0.45165339931227494 None
0.4477052567735745 None
0.4477052567735713 None
0.4516533993122748 None
0.4516533993122908 None
0.4516533993122783 None
0.45165339931229403 None
0.45554839402846736 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.3589258361692094 None
0.3517529726960299 None
0.34024372616421006 None
0.3424727992935695 None
0.3273484740336213 None
0.3290353957370658 None
0.3357251906420423 None
0.3279189447674703 None
0.3318138838559316 None
0.3318138838560613 None
0.3318138838589909 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.3480722205412848 None
0.32108901077602314 None
0.29405431448806957 None
0.26719776739859497 None
0.24471988597183328 None
0.21876276115296103 None
0.18945799783358508 None
0.16077988856377814 None
0.15180019784759524 None
0.1411627135295701 None
0.1351582658926172 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.34531633584905347 None
0.3183544425941973 None
0.2954230372429991 None
0.272700966342984 None
0.24246221160927361 None
0.21653485758437707 None
0.19113510664970282 None
0.1703523124677783 None
0.14344893569675055 None
0.1405952565113639 None
0.12673698236536343 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.3488273007525703 None
0.3179745422829954 None
0.2989845946623197 None
0.2685571310672928 None
0.24246179262153295 None
0.21298132258259345 None
0.19183449438834183 None
0.1674903451589837 None
0.14852292633569716 None
0.12928814489619678 None
0.12693192516140656 None
Done!


In [42]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d, 
                                                      rank_start = 1, 
                                                      rank_step = 1,
                                                      num_iter = 1000)

model_best.optimizer = None
saver([distance_best, model_best], data_path("diluted_fitting_4qubit_32layer.model"))

  0%|          | 0/1000 [00:00<?, ?it/s]

0.5259592882768112 None
0.5202041201413758 None
0.5173054131193121 None
0.5169367575737605 None
0.5168865588654987 None
0.5207765450952775 None
0.520776127212733 None
0.5168811397211148 None
0.5168811385784982 None
0.5207760999127549 None
0.5168811385435712 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.4204554912464412 None
0.4044448859764608 None
0.39963247750444386 None
0.39373261519852065 None
0.3911295415453419 None
0.38673015858348586 None
0.40241641247839655 None
0.3869430399798697 None
0.39478617723455706 None
0.38694303997976276 None
0.3908912103302758 None


  0%|          | 0/1000 [00:00<?, ?it/s]

...

0.4220555949085249 None
0.39882295651692995 None
0.3716480964717377 None
0.3407258417220768 None
0.31397603898267656 None
0.29156193258022384 None
0.26966354620498845 None
0.22846234268504798 None
0.21572284178919632 None
0.19696254035220376 None
0.19610425083128294 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.42371375658336974 None
0.4005474827056314 None
0.3695671536295553 None
0.3426040343721518 None
0.3120518211532069 None
0.29383399377973707 None
0.2646585792065765 None
0.2401839927086912 None
0.20877060227960556 None
0.197589430397304 None
0.18851782005592663 None


  0%|          | 0/1000 [00:00<?, ?it/s]

0.4254463010042312 None
0.3985748982126133 None
0.37180755290226525 None
0.34926012505020493 None
0.31943645214737043 None
0.29016032597232955 None
0.265433930088053 None
0.24547062709180908 None
0.22207846502112064 None
0.19587896235529678 None
0.1910411242146915 None
Done!
